In [4]:
import torch
import torch.nn as nn

In [5]:

def backward(tensors, grad_tensors=None, retain_graph=False, create_graph=False):
    if grad_tensors is None:
        grad_tensors = [None] * len(tensors)
    else:
        grad_tensors = list(grad_tensors)

    grad_tensors = _make_grads(tensors, grad_tensors)

    if create_graph and not retain_graph:
        retain_graph = True

    torch.autograd.backward(tensors, grad_tensors, retain_graph=retain_graph, create_graph=create_graph)


In [6]:
### CrEating the Model of the Neural Network
class MnistModel(nn.Module):
    ## We are creating feed forward neural network with one hidden layer
    def __init__(self , input_size, hidden_size, output_size):
        super().__init__()
        ## Hidden Layer
        self.Linear1 = nn.Linear(input_size, hidden_size)
        ## Output Layer
        self.Linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, xb):
        ## Flatten the image tensors
        xb = xb.view(xb.size(0), -1)
        ## We can also use xb = xb.reshape(xb.size(0), -1)
        ## Get intermediate outputs using hidden layer
        out = self.Linear1(xb)
        ## Apply activation function
        out = F.relu(out)
        ## Get predictions using output layer
        out = self.Linear2(out)
        return out

    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        ## Self is the model itself
        ## It will internally invoke the forward method andd will generate the output
        loss = F.cross_entropy(out, labels)
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss, 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

        

## In This Program actually we are defining a class all of these function will be used when we train the model i.e. when we define our fit function

In [7]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [9]:
def fit(epochs , lr, model , train_loader, val_loader , opt_function = torch.optim.Adam):
    history = []
    optimizer = opt_function(model.parameters(), lr)
    # We are mentionning for how meny epoches we need to run the weight update i.e. Traning
    for epoch in range(epochs):
        ## Traning Phase
        for batch in train_loader:
            ## Loading the data
            loass = model.training_step(batch)
            loss.backward()
            ## Calculating the gradients
            optimizer.step()
            ## wpdating the weights based upion the optimizers
            optimizer.zero_grad()
            ## Reset the gradients to zero
        ## Vlidatio Phase
        result  = evaulate(model, val_loader)
        model.epoch_end(epoch,result)
        history.append(result)
    return history
        